In [1]:
from random import randint, uniform

In [2]:
class Profile:

    def __init__(self, k, motifs = []):
        self.matrix = [{'A': 0, 'C': 0, 'G': 0, 'T': 0}.copy() for i in range(k)]
        self.k = k
        self.size = len(motifs)
        for motif in motifs:
            for i in range(self.k):
                self.matrix[i][motif[i]] += 1

    def add(self, motif):
        self.size += 1
        for i in range(self.k):
            self.matrix[i][motif[i]] += 1
                      
    def score(self, motif):
        value = 1.0
        for i in range(self.k):
            value *= (self.matrix[i][motif[i]] + 1) / (self.size + 2)
            
        return value


In [3]:
def most_probable(profile, string):
    best_score = 0.0
    k = profile.k
    mp_kmer = string[0:k]
    
    for i in range(len(string)-k+1):
        kmer = string[i:i+k]
        
        if profile.score(kmer) > best_score:
            best_score = profile.score(kmer)
            mp_kmer = kmer
    
    return mp_kmer

In [4]:
def score(motifs):
    k = len(motifs[0])
    p = Profile(k, motifs)
    
    value = 0
    for i in range(k):
        max_freq = max(p.matrix[i].values())
        cons_letter = 'A'
        for letter in p.matrix[i].keys():
            if p.matrix[i][letter] == max_freq: cons_letter = letter
                
        for j in range(len(motifs)):
            if motifs[j][i] != cons_letter:
                value += 1
                
    return value

In [5]:
def make_motifs(profile, dna):
    motifs = []
    for string in dna:
        motifs.append(most_probable(profile, string))
        
    return motifs       

In [6]:
def random_motif(string, k):
    pos = randint(0, len(string)-k)
    
    return string[pos:pos+k]

In [7]:
def random_dist_motif(string, profile):
    k = profile.k
    scores = [profile.score(string[i:i+k]) for i in range(len(string)-k+1)]  
    max_number = sum(scores)
    random_number = uniform(0, max_number)
 
    pos = -1
    while random_number >= 0:
        pos += 1
        random_number -= scores[pos]
         
    return string[pos:pos+k]

In [8]:
def GibbsSampler(dna, k, t, N):
    Motifs = [random_motif(string, k) for string in dna]
    bestMotifs = Motifs
    
    for j in range(N):
        i = randint(0, t-1)
        p = Profile(k, Motifs[0:i] + Motifs[i+1:])
        Motifs[i] = random_dist_motif(dna[i], p)
 
        if score(Motifs) < score(bestMotifs):
            bestMotifs = Motifs
            
    return bestMotifs

In [9]:
filename = '/content/rosalind_ba2g (3).txt'

In [10]:
with open(filename) as file:
    k, t, N = [int(x) for x in file.readline().split()]
    dna = []
    for line in file:
        dna.append(line.rstrip())

In [11]:
from tqdm import tqdm

In [12]:
bestMotifs = GibbsSampler(dna, k, t, N)

for i in tqdm(range(20 - 1)):
    motifs = GibbsSampler(dna, k, t, N)
    if score(motifs) < score(bestMotifs):
        bestMotifs = motifs

100%|██████████| 19/19 [00:56<00:00,  2.96s/it]


In [13]:
for motif in bestMotifs:
    print(motif)

AGCATAGGCTAAGCA
AGCTCAGGCAATAGA
AGCAACACCAATAGA
AGCAAGGGTTGTAGA
AGCACCTGCAATAGA
TCGAAGGGCAATAGA
AGCGAAGTAAATAGA
GGCAAGGGCAATATG
AGCACTAGCAATAGA
AAAGAGGGCAATAGA
AGCAAGGTTCATAGA
AGCAAGGGCACCTGA
AGCAAGGGCAATCAT
AGCAAGTAGAATAGA
AGCCTTGGCAATAGA
AGCAACACCAATAGA
AGCAAGGGCAACTCA
AGCAAGGGCTGCAGA
AGCAAGTTGAATAGA
AGACCGGGCAATAGA
